<a href="https://colab.research.google.com/github/Cnaraa/websiteClassification/blob/main/Preparation_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import pandas as pd
import numpy as np

file_path = "/content/drive/MyDrive/Website Classification/dataset/translated_data.csv"
df = pd.read_csv(file_path)

In [42]:
df.head()

,translated_title,translated_summary,categories
0,"Представительство, функциональность и натурали...",В статье анализируются основополагающие принци...,"Искусственный интеллект, Робототехника"
1,NeuroChat: Нейродорожный AI Chatbot для адапта...,"В то же время, электроэнцефалические (ЭЕГ) ней...","Человеко-компьютерное взаимодействие, Искусств..."
2,Денонсация Гамильтонской сети физических обосн...,Системы машинного обучения для физических проб...,"Машинное обучение, Искусственный интеллект"
3,"Сначала снимки фильтров, затем генерировать ин...",Визуальная инструктирование (VIT) для моделей ...,"Компьютерное зрение и распознавание образов, И..."
4,Когда модель Большого Видения и Лангуажа совпа...,Эффективное понимание с помощью языка зрения к...,"Компьютерное зрение и распознавание образов, И..."


In [43]:
df = df.drop_duplicates(subset='translated_title', keep='first')
df = df.drop_duplicates(subset='translated_summary', keep='first')
df = df.dropna(subset=['translated_title'])
df = df.dropna(subset=['translated_summary'])

In [44]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [45]:
import re
from nltk.corpus import stopwords
from pymystem3 import Mystem

mystem = Mystem()
stop_words = set(stopwords.words('russian'))
cached_lemmatize = {}

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'[^а-яёa-z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    words = [word for word in text.split() if word not in stop_words]

    def lemmatize_word(word):
        if word in cached_lemmatize:
            return cached_lemmatize[word]
        if re.match(r'^[а-яё]+$', word):
            lemma = mystem.lemmatize(word)[0]
            cached_lemmatize[word] = lemma
            return lemma
        return word

    lemmas = [lemmatize_word(word) for word in words]
    return " ".join(lemmas).strip()


In [46]:
df['preprocessed_translated_title'] = df['translated_title'].apply(preprocess_text)
df['preprocessed_translated_summary'] = df['translated_summary'].apply(preprocess_text)

In [47]:
df['text'] = df['preprocessed_translated_title'] + " " + df['preprocessed_translated_summary']

In [49]:
df_preparation = df[['preprocessed_translated_title', 'preprocessed_translated_summary', 'text', 'categories']].copy()

In [50]:
df_preparation.head()

,preprocessed_translated_title,preprocessed_translated_summary,text,categories
0,представительство функциональность натуралисти...,статья анализироваться основополагающий принци...,представительство функциональность натуралисти...,"Искусственный интеллект, Робототехника"
1,neurochat нейродорожный ai chatbot адаптация у...,время электроэнцефалический эег нейроадаптивны...,neurochat нейродорожный ai chatbot адаптация у...,"Человеко-компьютерное взаимодействие, Искусств..."
2,денонсация гамильтонский сеть физический обосн...,система машинный обучение физический проблема ...,денонсация гамильтонский сеть физический обосн...,"Машинное обучение, Искусственный интеллект"
3,сначала снимок фильтр затем генерировать инстр...,визуальный инструктирование vit модель большой...,сначала снимок фильтр затем генерировать инстр...,"Компьютерное зрение и распознавание образов, И..."
4,модель большой видение лангуаж совпадать больш...,эффективный понимание помощь язык зрение крупн...,модель большой видение лангуаж совпадать больш...,"Компьютерное зрение и распознавание образов, И..."


In [ ]:
output_path = ""
df_preparation.to_csv(output_path, index=False)